In [1]:
%reload_ext autoreload
%autoreload 2

In [5]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [7]:
from parsers import arbitrary, text_utils
import artm
import hierarchy_utils
import pickle
from algo.arranging.api import arrange_topics
from algo.arranging.crossmin import CrossMinimizer
import numpy as np

In [9]:
%%time

MODEL_PATH = "hartm/"

artm_model = hierarchy_utils.hARTM(theta_columns_naming="title",
                                   cache_theta=True)
artm_model.load(MODEL_PATH)


CPU times: user 1min 56s, sys: 5.26 s, total: 2min 1s
Wall time: 1min 53s


In [10]:
artm_extra_info = pickle.load(open(MODEL_PATH + "/extra_info.dump", "rb"))

In [11]:
# Extract Phi, Psi and Theta matrices
phis = []
psis = []
#theta = artm_extra_info["theta"]
# theta = pickle.load(open(THETA_MODEL_PATH, "rb"))["theta"]
for level_idx, artm_level in enumerate(artm_model._levels):
    phis.append(artm_level.get_phi(class_ids="flat_tag"))
    if level_idx > 0:
        psis.append(artm_level.get_psi())
        
phi0_topic_titles = list(filter(lambda x: x.startswith("topic"), phis[0].columns))
phi1_topic_titles = list(filter(lambda x: x.startswith("topic"), phis[1].columns))

In [12]:
new_phi1_topic_order = np.array(phi1_topic_titles)[arrange_topics(phis[1][phi1_topic_titles].values)]
phis[1] = phis[1][new_phi1_topic_order]
psis[0] = psis[0].loc[new_phi1_topic_order]

In [13]:
children_threshold = 0.05

D = np.zeros((len(phi0_topic_titles), len(phi1_topic_titles)))

for parent_id, parent in enumerate(phi0_topic_titles):
    for child_id, maybe_child in enumerate(phi1_topic_titles):
        if psis[0].loc[maybe_child, parent] > children_threshold:
            D[parent_id, child_id] = 1

In [14]:
cm = CrossMinimizer(D)
idx = cm.solve(mode="split10N", model=None)
new_phi0_topic_order = np.array(phi1_topic_titles)[idx]

In [15]:
idx

array([13,  8, 11,  2, 12, 17,  7, 14,  5,  6, 16,  3,  1, 10,  4, 15,  9,
       18, 19,  0])

In [16]:
new_phi0_topic_order

array(['topic_13', 'topic_8', 'topic_11', 'topic_2', 'topic_12',
       'topic_17', 'topic_7', 'topic_14', 'topic_5', 'topic_6', 'topic_16',
       'topic_3', 'topic_1', 'topic_10', 'topic_4', 'topic_15', 'topic_9',
       'topic_18', 'topic_19', 'topic_0'], 
      dtype='<U8')

In [17]:
new_phi1_topic_order

array(['topic_49', 'topic_0', 'topic_38', 'topic_55', 'topic_16',
       'topic_26', 'topic_18', 'topic_20', 'topic_10', 'topic_34',
       'topic_51', 'topic_28', 'topic_14', 'topic_22', 'topic_19',
       'topic_25', 'topic_30', 'topic_44', 'topic_23', 'topic_11',
       'topic_32', 'topic_54', 'topic_29', 'topic_2', 'topic_5',
       'topic_27', 'topic_8', 'topic_47', 'topic_13', 'topic_52',
       'topic_40', 'topic_42', 'topic_21', 'topic_24', 'topic_56',
       'topic_43', 'topic_12', 'topic_9', 'topic_31', 'topic_37',
       'topic_7', 'topic_46', 'topic_4', 'topic_36', 'topic_17', 'topic_3',
       'topic_15', 'topic_39', 'topic_59', 'topic_50', 'topic_41',
       'topic_6', 'topic_58', 'topic_1', 'topic_45', 'topic_53',
       'topic_57', 'topic_35', 'topic_33', 'topic_48'], 
      dtype='<U8')

In [20]:
def get_topic_name(lid, tid):
    return ", ".join(phis[lid][tid].sort_values()[-3:][::-1].index)

In [54]:
print("\n".join(map(lambda t: t + " \t" + get_topic_name(0, t), new_phi0_topic_order)))

topic_17 	христианство, средневековье, религия
topic_7 	медицина, онкология, биомедицина
topic_6 	биология, генетика, эволюция
topic_19 	право, юриспруденция, римское_право
topic_2 	физика, физика_элементарных_частиц, квантовая_физика
topic_12 	социология, город, урбанистика
topic_18 	россия, украина, география
topic_8 	мозг, психология, нейробиология
topic_0 	математика, кибернетика, логика
topic_13 	культура, массовая_культура, культурология
topic_9 	экономика, сша, авторское_право
topic_11 	политика, государство, политология
topic_1 	технологии, искусственный_интеллект, информационные_технологии
topic_14 	образование, наука, история_науки
topic_16 	философия, политическая_философия, древняя_греция
topic_4 	земля, экология, солнечная_система
topic_3 	химия, нанотехнологии, материаловедение
topic_10 	история, история_россии, европа
topic_5 	астрономия, астрофизика, вселенная
topic_15 	язык, лингвистика, филология


In [55]:
print("\n".join(map(lambda t: t + " \t" + get_topic_name(0, t), phi0_topic_titles)))

topic_0 	математика, кибернетика, логика
topic_1 	технологии, искусственный_интеллект, информационные_технологии
topic_2 	физика, физика_элементарных_частиц, квантовая_физика
topic_3 	химия, нанотехнологии, материаловедение
topic_4 	земля, экология, солнечная_система
topic_5 	астрономия, астрофизика, вселенная
topic_6 	биология, генетика, эволюция
topic_7 	медицина, онкология, биомедицина
topic_8 	мозг, психология, нейробиология
topic_9 	экономика, сша, авторское_право
topic_10 	история, история_россии, европа
topic_11 	политика, государство, политология
topic_12 	социология, город, урбанистика
topic_13 	культура, массовая_культура, культурология
topic_14 	образование, наука, история_науки
topic_15 	язык, лингвистика, филология
topic_16 	философия, политическая_философия, древняя_греция
topic_17 	христианство, средневековье, религия
topic_18 	россия, украина, география
topic_19 	право, юриспруденция, римское_право


In [25]:
phi0_flat_spectrum = np.array(phi0_topic_titles)[arrange_topics(phis[0][phi0_topic_titles].values)]

In [26]:
print("\n".join(map(lambda t: t + " \t" + get_topic_name(0, t), phi0_flat_spectrum)))

topic_13 	культура, литература, массовая_культура
topic_15 	язык, лингвистика, филология
topic_19 	право, юриспруденция, римское_право
topic_17 	христианство, религия, китай
topic_10 	история, история_россии, средневековье
topic_18 	россия, сша, европа
topic_9 	экономика, сша, авторское_право
topic_11 	политика, государство, политология
topic_12 	социология, общество, город
topic_14 	образование, наука, история_науки
topic_16 	философия, политическая_философия, история_философии
topic_8 	психология, мозг, нейробиология
topic_6 	биология, эволюция, генетика
topic_7 	медицина, онкология, биомедицина
topic_0 	математика, нейронные_сети, искусственный_интеллект
topic_1 	технологии, информационные_технологии, интернет
topic_3 	химия, нанотехнологии, материаловедение
topic_2 	физика, физика_элементарных_частиц, квантовая_физика
topic_5 	астрономия, астрофизика, вселенная
topic_4 	земля, экология, солнечная_система


In [22]:
# Двухуровневый спектр
pickle.dump((new_phi0_topic_order, new_phi1_topic_order, D), open("spectrum.dump", "wb"))

In [27]:
# Одноуровневый спектр
level_0_topics_ids = list(map(lambda tid: "level_0_%s" % tid, phi0_flat_spectrum))
artm_extra_info["spectrums"] = [level_0_topics_ids]
with open(MODEL_PATH + "/extra_info.dump", "wb") as fout:
    pickle.dump(artm_extra_info, fout)